In [1]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
import simplejson as json
import re 
pd.set_option('display.max_rows', 1000) 
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
columns = ['Name', 'Department', 'Title', 'Regular', 'Retro', 'Other', 'Overtime', 'Injury', 'Detail', 'Quinn', 'Total', 'Postal']
earnings_df = pd.DataFrame(columns = columns)
earnings_df['Year'] = np.nan

for year in range(2011, 2022):
    
    df_for_year = pd.read_csv('./data/salaries_' + str(year) + '.csv', encoding='latin1')
    
    #retrieve only the first part of each column name 
    columns = {col: (re.split('[^a-zA-Z]', col.strip())[0]).title() for col in df_for_year.columns}
    
    #standardize column names
    df_for_year.rename(columns=columns, inplace=True)
    df_for_year.rename(columns={"Injured": "Injury"}, inplace=True)
    df_for_year['Year'] = year
  
    earnings_df = pd.concat([earnings_df, df_for_year], sort=False, axis=0)

In [3]:
earnings_df.sample(100)

,Name,Department,Title,Regular,Retro,Other,Overtime,Injury,Detail,Quinn,Total,Postal,Year,Zip,Details
13550,"Hale,Nyerrie Leon",UP Academy Dorchester,Cluster Substitute ES,"$3,439.21",NaN,NaN,NaN,NaN,NaN,NaN,"$3,439.21",2111,2017.00,NaN,NaN
3816,"Oakes,Robert C",BPS Boston Latin,Teacher,"$100,235.75",NaN,NaN,NaN,NaN,NaN,NaN,"$100,235.75",2131,2017.00,NaN,NaN
4676,"Daniels,Haven Sarah Ripley",BPS Perkins Elementary,Teacher,$107936.94,$0.00,$10830.00,$0.00,$0.00,NaN,$0.00,$118766.94,02126,2015.00,NaN,$0.00
12607,"Powell,Amanda Catherine",Boston Public Schools,Sub Lunch Monitor,$368.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$368.00,NaN,2012.00,02127-2015,NaN
778,"Miller,William L",Boston Fire Department,Fire Lieutenant,"141,104.13","13,002.85",550.00,"19,938.26",502.65,"23,737.50",-,"198,835.39",2124,2019.00,NaN,NaN
9893,"Irizarry,Christian",Boston Public Schools,Community Field Coordinator,$60225.14,$524.77,$15000.00,$0.00,$0.00,$0.00,$0.00,$75749.91,NaN,2013.00,02131-4520,NaN
4913,"Bzdula,Patrick S",BPS Madison Park High,Teacher,"71,535.24","2,778.64",585.12,NaN,NaN,NaN,NaN,"74,899.00",01701,2018.00,NaN,NaN
9467,"Rivera,Raquel",BPS Mozart Elementary,Sub Lunch Monitor,$24.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$24.00,NaN,2014.00,02210,NaN
15402,"Oliver,Elyse Tia Desiree",UP Academy Holland,Paraprofessional,"$7,383.87",NaN,NaN,"$1,138.35",NaN,NaN,NaN,"$8,522.22",2119,2016.00,NaN,NaN
21820,"McGoff,Cullen Basil",Public Works Department,Laborer Seasonal (PW),"11,601.11",NaN,NaN,361.83,NaN,NaN,NaN,"11,962.94",02121,2018.00,NaN,NaN


In [4]:
#targeted (hard-coded) cleaning for specific row
earnings_df = earnings_df.loc[earnings_df['Department']!='DEPARTMENT_NAME']

facts = ['Regular', 'Retro', 'Other', 'Overtime', 'Injury', 'Detail', 'Quinn', 'Total']

facts_transforms = {
    '^-$|^None$|^nan$|\)':0, 
    '':0,
    ',':'', 
    '\$':'', 
    ' ':'', 
    '^\(':'-'
}

#convert facts to useable types
earnings_df['Year'] = earnings_df['Year'].astype(int)
earnings_df[facts] = earnings_df[facts].astype(str).applymap(lambda x: x.strip())
earnings_df[facts] = earnings_df[facts].replace(facts_transforms, regex=True)
earnings_df[facts] = earnings_df[facts].astype(float)
earnings_df[facts].fillna(0, inplace=True)

#combine 'Detail' and 'Quinn' with 'Other'
earnings_df['Other'] = earnings_df[['Other', 'Detail', 'Quinn']].sum(axis=1)

earnings_df.drop(columns=['Detail', 'Quinn', 'Zip', 'Details'], inplace=True)

c:\users\cpsch\gitrepos\bostonsalaries\venv\lib\site-packages\pandas\core\frame.py:4147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [5]:
earnings_df.sample(100)

,Name,Department,Title,Regular,Retro,Other,Overtime,Injury,Total,Postal,Year
1636,"James,Julie K",Boston Police Department,Criminalist IV,105377.49,0.00,2026.49,17391.62,0.00,124795.60,2186,2017
5339,"Leon,Brigido Orlando",Boston Police Department,Police Officer,68181.24,0.00,19184.00,19251.17,0.00,106616.41,NaN,2011
13332,"DiNunno,Dianna H",BPS Adams Elementary,Lunch Hour Monitors,7647.99,0.00,17.50,0.00,932.46,8597.95,2128,2017
14853,"Murat,Lamercy",Boston Public Schools,Part-Time Custodian,11383.34,329.79,0.00,0.00,0.00,11713.13,NaN,2013
19603,"Bufi,Kristaq",Boston Public Library,Jr Building Custodian,43388.06,0.00,300.00,17253.53,0.00,60941.59,01867,2018
2909,"Murrell,Les E",Boston Fire Department,Fire Fighter,117802.75,9465.71,3184.33,8961.36,691.90,140106.05,2131,2019
5231,"Grammer,Christopher J.",BPS Boston Latin,Teacher,117001.02,0.00,1600.00,0.00,0.00,118601.02,02132,2020
11247,"Johnson,Alissa Michelle",BPS Orchard Gardens K-8 Pilot,Teacher,76277.44,0.00,0.00,0.00,0.00,76277.44,02125,2020
6684,"Clemons Sr.,David J",Boston Police Department,Police Officer Hdq Dispatch,101324.72,0.00,10932.60,547.70,0.00,112805.02,02121,2021
1238,"Luciano,Bienvenido",Boston Public Library,Sr Bldg Custodian,47084.10,0.00,667.81,1010.49,0.00,48762.40,NaN,2014


In [6]:
dimensions = ['Name', 'Department', 'Title', 'Postal']
earnings_df[dimensions] = earnings_df[dimensions].astype(str)

In [7]:
#purpose: clean Postal column

#add a 0 to the front of any code with 4 digits
mask = earnings_df['Postal'].str.len() == 4
earnings_df.loc[mask, 'Postal'] = '0' + earnings_df.loc[mask, 'Postal']

#remove delivery route number
earnings_df['Postal'] = earnings_df['Postal'].str.split('-', expand=True)[0]

#set postal codes with non-numeric characters to UNKNOWN
earnings_df.loc[earnings_df['Postal'].str.match('[^0-9]', na=False), 'Postal'] = 'UNKNOWN'

#hard cleaning for specific rows
postal_codes = {
    'Ostiguy,David M': '02327',
    'Karales,George Alfred': '02170',
    'Smith,Kenneth J': '02124',
    'Thomas,Sarita J': '02125',
    'Morris,Judith A.': '02170',
    'Mendez,Jose R': '02135',
    'Morrison,June': '02481'
}

for name, code in postal_codes.items():
    earnings_df.loc[earnings_df['Name'].str.match(name), 'Postal'] = code


In [8]:
#purpose: clean Name column
earnings_df['Name'] = earnings_df['Name'].replace({'\.':''}, regex=True).str.upper()

name = earnings_df['Name'].str.split(',', n=1, expand=True)
earnings_df['Last']=name[0]
earnings_df['First']=name[1]
earnings_df.drop(columns=["Name"], inplace=True) 

dimensions = ['First', 'Last', 'Department', 'Title', 'Postal']

In [9]:
#return list of all unique values for a given column sorted in alphabetical order
def show_unique(column):
    optns = earnings_df[column].unique()
    optns.sort()
    return optns

#purpose: get pivot table of employee counts by department
def count_by_group(column, rec=r'(.*?)'):
    
    group_counts = earnings_df.groupby([column, 'Year'])['Total'].count().reset_index(name="count")
    group_counts_table = pd.pivot_table(group_counts, values='count', index=column, columns='Year', aggfunc='sum', fill_value=0, margins=True).reset_index()
    
    #get rid of the calculated sums by row, doesn't make sense for time series data
    return group_counts_table.iloc[:,0:-1].loc[group_counts_table[column].str.match(rec)]

In [10]:
#create uniform spacing, convert to all caps, and remove unwanted characters

title_cleaning = {
    '\.':'', 
    '(?<=[a-z])([A-Z])':r' \1', 
    '\(':' (', 
    '\)':') ', 
    '\\\\':'', 
    '&':' AND ', 
    '\,':'/', 
    '\#':'', 
    '\s+':' '
}

earnings_df['Title'] = earnings_df['Title'].replace(title_cleaning, regex=True).str.upper()
earnings_df[dimensions] = earnings_df[dimensions].applymap(lambda x: x.strip() if x else x)

In [11]:
#preliminary cleaning for Title column to unstick words that were stuck together
title_cleaning = {
    '([A-Z])(OF)([\/\s\)]|$)':r'\1 OF ', 
    '([A-Z])(BPD)':r'\1 \2', 
    '(BPD)([A-Z])':r'\1 \2'
}

earnings_df['Title'].replace(title_cleaning, regex=True, inplace=True)

In [12]:
#replace abbreviations and spelling errors with proper words in title column

#CHALLENGES --> 
#OP/OPER: OPERATOR VS OPERATIONS
#SP/SPEC: SPECIAL VS SPECIALIST
#SERV/SVC: SERVICE VS SERVICES
#COM/COMM: COMMUNITY VS COMMUNICATIONS VS COMMISSIONER VS COMMISSION VS COMMITTEE

title_transforms = {
    'ADMIN AND FINANCE':'ADMINISTRATION AND FINANCE',
    'ADMIN/FINANCE':'ADMINISTRATION AND FINANCE',
    'OFFC':'OFFICER',
    'OFFCR':'OFFICER',
    'SEN':'SENIOR',
    'DET':'DETECTIVE',
    'SUP':'SUPERVISOR',
    'SUPV':'SUPERVISOR',
    'SPV':'SUPERVISOR',
    'EXEC':'EXECUTIVE',
    'AN':'ANALYST',
    'ANL':'ANALYST',
    'ANAL':'ANALYST',
    'TECH':'TECHNICIAN',
    'DEPT':'DEPARTMENT',
    'EQUIP':'EQUIPMENT',
    'EQUIPMENT OPER':'EQUIPMENT OPERATOR',
    'ALARM OPER':'ALARM OPERATOR',
    'METER OPER':'METER OPERATOR',
    'COMPUTER OPER':'COMPUTER OPERATOR',
    'PROJ':'PROJECT',
    'SP':'SPECIAL',
    'STFF':'STAFF',
    'ACAD':'ACADEMY',
    'INSTR':'INSTRUCTOR',
    'ASST':'ASSISTANT',
    'ASSIST':'ASSISTANT',
    'ASS':'ASSISTANT',
    'DEP':'DEPUTY',
    'SUPN':'SUPERINTENDENT',
    'SYS':'SYSTEMS',
    'COOR':'COORDINATOR',
    'COORD':'COORDINATOR',
    'SEC':'SECRETARY',
    'LIEUT':'LIEUTENANT',
    'LT':'LIEUTENANT',
    'MAINT':'MAINTENANCE',
    'MAIN':'MAINTENANCE',
    'DIR':'DIRECTOR',
    'MGMT':'MANAGEMENT',
    'MGR':'MANAGER',
    'MNGR':'MANAGER',
    'MANGR':'MANAGER',
    'MED':'MEDICAL',
    'PROC':'PROCESSING',
    'CORP':'CORPORATION',
    'ASSOC':'ASSOCIATE',
    'ASSESS OPER MANAGEMENT':'ASSESSING OPERATIONS MANAGEMENT',
    'OPER':'OPERATIONS',
    'OP':'OPERATOR',
    'INC COMM':'INCIDENT COMMAND',
    'COMM SERV':'COMMUNITY SERVICE',
    'COMM OUTREACH':'COMMUNITY OUTREACH',
    'ASSISTANT COMM':'ASSISTANT COMMISSIONER',
    'DEPUTY COMM':'DEPUTY COMMISSIONER',
    'COMM OFFICE':'COMMISSIONERS OFFICE',
    'RADIO COMM':'RADIO COMMUNICATIONS',
    'COMM EQUIPMENT':'COMMUNICATIONS EQUIPMENT',
    'COMMUNIC':'COMMUNICATIONS',
    'COMMUN':'COMMUNICATIONS',
    'COMM LEADER':'COMMUNITY LEADER',
    'FIRE COMM':'FIRE COMMISSIONER',
    'HOUSING COMM':'HOUSING COMMISSION',
    'SCHOOL COMM':'SCHOOL COMMITTEE',
    'COMM AND INTERG':'COMMUNITY AND INTERGOVERNMENTAL',
    'BLDG':'BUILDINGS',
    'BLDGS':'BUILDINGS',
    'BDG':'BUILDINGS',
    'REG':'REGISTRAR',
    'SERV':'SERVICE',
    'SVC':'SERVICE',
    'SRV':'SERVICE',
    'PRIN':'PRINCIPAL',
    'PARA':'PARAPROFESSIONAL',
    'DIST':'DISTRICT',
    'FF':'FIRE FIGHTER',
    'A AND F':'ADMINISTRATION AND FINANCE',
    'F':'FIRE',
    'INSTRUC':'INSTRUCTOR',
    'SR':'SENIOR',
    'JR':'JUNIOR',
    'MECH':'MECHANIC',
    'MECHA':'MECHANIC',
    'MACH':'MACHINE',
    'GEN':'GENERAL',
    'ADMN':'ADMIN',
    'ADM':'ADMIN',
    'ENG':'ENGINEER',
    'STRUCT':'STRUCTURAL',
    'FRPRS':'FOREPERSON',
    'FRPR':'FOREPERSON',
    'FOREPRS':'FOREPERSON',
    'CONST':'CONSTRUCTION',
    'LBR':'LABORER',
    'RPR':'REPAIR',
    'REP':'REPAIR',
    'SPEC':'SPECIAL',
    'INCT':'INCIDENT',
    'COMND':'COMMAND',
    'FIN COM':'FINANCE COMMISSION',
    'MAS':'MASTER',
    'RPPRS':'REPAIRPERSON',
    'REPPRS':'REPAIRPERSON',
    'REPRPRS':'REPAIRPERSON',
    'REPAIRPR':'REPAIRPERSON',
    'REPAIRPRS':'REPAIRPERSON',
    'RPPR':'REPAIR PERSON',
    'WKG':'WORKING',
    'PW':'PUBLIC WORKS',
    'P W':'PUBLIC WORKS',
    'HVY':'HEAVY',
    'MTR':'MOTOR',
    'INSP':'INSPECTOR',
    'INSPEC':'INSPECTOR',
    'TRA':'TRAFFIC',
    'OPR':'OPERATIONS',
    'MEO':'MOTOR EQUIPMENT OPERATOR',
    'CFM':'(CFM)',
    'ELEC':'ELECTRIC',
    'EQUI':'EQUIPMENT',
    'COLL TRS':'COLLECTOR TREASURER',
    'COLL-TRS':'COLLECTOR TREASURER',
    'ACNTNG':'ACCOUNTING',
    'CRFTSPRS':'CRAFTSPERSON',
    'COMMSS':'COMMISSIONER',
    'COMR':'COMMISSIONER',
    'COMMIS':'COMMISSIONER',
    'CFM':'',
    'COUNSLR':'COUNSELOR',
    'BD':'BOARD',
    'MEMBER BOARD':'MEMBER OF BOARD',
    'LIB':'LIBRARIAN',
    'LIBR':'LIBRARIAN',
    'LIBRARIN':'LIBRARIAN',
    'SVCS':'SERVICES',
    'CAMP JO':'(CAMP JOY)',
    'CAM JO':'(CAMP JOY)',
    'SER':'SERVICES',
    'PROT':'PROTECTIVE',
    'REL':'RELATIONS',
    'SUPVISING':'SUPERVISING',
    'PROP':'PROPERTY',
    'DISP':'DISPATCHER',
    'CHF':'CHIEF',
    'PMDGRAFF REMOVAL':'(GRAFFITI REMOVAL)',
    'PAINT':'PAINTER',
    'ENFORCE':'ENFORCEMENT',
    'DEVELOP':'DEVELOPMENT',
    'DEVEL':'DEVELOPMENT',
    'PROG':'PROGRAM',
    'PWD':'',
    'SWIM':'SWIMMING',
    'REGNL':'REGIONAL',
    'ACCTNG':'ACCOUNTING',
    'ACCT':'ACCOUNTING',
    'ENGR':'ENGINEER',
    'EQU':'EQUIPMENT',
    'EQ':'EQUIPMENT',
    'ANIM CNTL':'ANIMAL CONTROL',
    'OFCR':'OFFICER',
    'CLRK':'CLERK',
    'PARKS AND REC':'PARKS AND RECREATION',
    'P AND R':'PARKS AND RECREATION',
    '\(PARK\)':'(PARKS AND RECREATION)',
    'SPC':'SPECIAL',
    'HDQ':'HEADQUARTER',
    'TRANS':'TRANSPORTATION',
    'DISPCH':'DISPATCHER',
    'SUB':'SUBSTITUTE',
    'CUST':'CUSTODIAN',
    'NEIGH':'NEIGHBORHOOD',
    'YTH':'YOUTH',
    'HE':'',
    'BE':'',
    'FGR PRT EV':'FINGERPRINT EVIDENCE',
    'CH':'CHIEF',
    'SUM SCH':'SUMMER SCHOOL',
    'COM SCH':'COMMUNITY SCHOOL',
    'FAM':'FAMILY',
    'BPDFLEET':'BPD FLEET',
    'IBPDFLEET':'I (BPD FLEET',
    'IIBPDFLEET':'II (BPD FLEET',
    'EVIDENC':'EVIDENCE',
    'TECHNCN':'TECHNICIAN',
    'TCH':'TECHNICIAN',
    'SP ED':'SPECIAL ED',
    'SPED':'SPECIAL ED',
    'DP':'DATA PROCESSING',
    'PREV':'PREVENTION',
    'SCUBA DIV':'SCUBA DIVER',
    'DIV':'DIVISION',
    'POL':'POLICE',
    'OFFR':'OFFICER',
    'CLASSIFICATN':'CLASSIFICATION',
    'ATTN':'ATTENDANT',
    'CAFE':'CAFETERIA',
    'SCHL':'SCHOOL',
    'SCH':'SCHOOL',
    'HOSP':'HOSPITAL',
    'ASSTO':'ASSISTANT TO',
    'CHIEFOF':'CHIEF OF',
    'EXC':"EXECUTIVE",
    'OFF':'OFFICE',
    'SRGT':'SERGEANT',
    'RET':'RETIREMENT',
    'RETIRE':'RETIREMENT',
    'RETIREME':'RETIREMENT',
    'RECYCLE':'RECYCLING',
    'REAS':'RESEARCH',
    'INVESTNS':'INVESTIGATIONS',
    'INVEST':'INVESTIGATOR',
    'HACKNEY':'HACKNEY UNIT',
    'TOTHE':'TO THE',
    'PRGS':'PROGRAMS',
    'JOURNEYPRS':'JOURNEYPERSON',
    'BLD':'BUILDINGS',
    'PLMG AND GSFTG':'PLUMBING AND GASFITTING',
    'PLG AND GAS FTNG':'PLUMBING AND GASFITTING',
    'EL IN AND MNT':'ELECTRICAL INSPECTION AND MAINTENANCE',
    'AL':'ALTERATION',
    'ALT':'ALTERATION',
    'CLK':'CLERK',
    'ADV':'ADVANCED',
    'MAST\)':'MASTER',
    'ADR':'(RETIRED - ADR)',
    'LIBRRIN':'LIBRARIAN',
    'SPECL':'SPECIALIST',
    'LAB RELATIONS':'LABOR RELATIONS',
    'C AND REPAIR':'CONSTRUCTION AND REPAIR',
    'CONSTR':'CONSTRUCTION',
    'PMDCONST':'PMD CONSTRUCTION',
    'AUD':'AUDITOR',
    'ENVRNMNTL':'ENVIRONMENTAL',
    'PR OF':'PARAPROFESSIONAL',
    'HOT LINE':'HOTLINE',
    'FCOMMISSIONER':'FIRE COMMISSIONER',
    'PROFESS':'PROFESSIONAL',
    'ARCHIV': 'ARCHIVE',
    'TECHG':'TECHNOLOGY',
    'FOREPR': 'FOREPERSON',
    'LIBRAY': 'LIBRARY',
    'SRVC': 'SERVICES',
    'MGNT': 'MANAGEMENT',
    'DEV': 'DEVELOPMENT'
    
}

#to prevent words within words from accidentally being changed, lets ensure that values from above dictionary must be
#between certain characters to change
for key, val in title_transforms.items():
    val = ' ' + val + ' '
    new_key = '(^|[\s\(\)\-\/])' + key + '([\s\(\)\-\/]|$)'
    earnings_df['Title'] = earnings_df['Title'].replace(new_key, val, regex=True).str.strip()

earnings_df['Title'].replace({'SPECIAL$':'SPECIALIST'}, regex=True, inplace=True)

In [13]:
count_by_group('Title').sort_values(2021, ascending=False).head(1000)

Year,Title,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
2168,All,20509,21140,22469,22233,21902,22046,22245,23603,23312,21858,22552
2060,TEACHER,5121,5249,5512,5426,5375,5363,5437,5541,5255,5106,5171
1378,PARAPROFESSIONAL,1108,1161,1209,1113,1180,1187,1215,1416,1356,1394,1509
1427,POLICE OFFICER,1281,1231,1308,1349,1273,1279,1331,1334,1346,1355,1321
926,FIRE FIGHTER,804,816,861,873,815,837,795,792,820,790,788
1978,SUBSTITUTE TEACHER,770,743,1045,837,793,771,759,983,724,534,505
1391,PART-TIME CAFETERIA ATTENDANT,360,389,417,360,356,341,334,375,374,372,405
1228,LUNCH HOUR MONITORS,388,368,424,384,377,370,360,392,341,319,330
1419,POLICE DETECTIVE,281,292,290,286,286,330,298,296,309,298,297
294,CAB MONITOR,252,319,379,343,354,365,306,232,337,264,288


In [14]:
show_unique('Department')

array(['ASD Graphic Arts', 'ASD Human Resources',
       'ASD Intergvernmtl Relations', 'ASD Office Of Labor Relation',
       'ASD Office of Budget Mangmnt', 'ASD Purchasing Division',
       'Accountability', 'Achievement Gap', 'Administration and Finance',
       'Advancement & Ext. Affairs', 'Age Strong',
       'Alighieri Montessori School', 'Arts & Cultural Development',
       'Assessing Department', 'Asst Superintendent-Network A',
       'Asst Superintendent-Network B', 'Asst Superintendent-Network C',
       'Asst Superintendent-Network D', 'Asst Superintendent-Network E',
       'Asst Superintendent-Network F', 'Asst Superintendent-Network G',
       'Auditing Department', 'BPS Adams Elementary',
       'BPS Adult Education', 'BPS Alternative Education',
       'BPS Alternative Education HS', 'BPS Another Course To Colleg',
       'BPS Athletics', 'BPS Bates Elementary',
       'BPS Beethoven Elementary', 'BPS Bi-Weekly 26 Leave',
       'BPS Blackstone Elementary', 'BPS Bos

In [15]:
# create uniform spacing, convert to all caps, and remove unwanted characters
cleaning_dict = {
    '\.':'', 
    '(?<=[a-z])([A-Z])':r' \1', 
    '\/':' AND ', 
    '\\\\':'', 
    '&':' AND ', 
    '\,':' ', 
    '\#':''
}
earnings_df['Department'] = earnings_df['Department'].replace(cleaning_dict, regex=True).str.upper()
earnings_df['Department'] = earnings_df['Department'].replace('\s+', ' ', regex=True).str.strip()

In [16]:
# move BPS programs to new 'Program' column and set dept to Boston Public Schools
bps_phrases = [
    '^BPS',
    'ELEMENTARY',
    'ACADEMY',
    'K-8',
    'MIDDLE',
    'HIGH'
    'SCHOOL',
    'ACAD$',
    'PILOT',
    ' EEC$',
    ' ELC$',
    ' EES$',
    '9-12',
    'ACHIEVEMENT GAP',
    'STUDENT',
    'SUPERINTENDENT',
    'CHIEF ACADEMIC OFFICER',
    'ENGLISH LANGUAGE LEARN',
    'ACCOUNTABILITY',
    '^ADVANCEMENT',
    'ENROLLMENT',
    '^EARLY LEARNING',
    '^HPEC',
    '^INFO AND INSTR',
    'TEACHING',
    'CAREER AND TECHNICAL ED',
    'CHIEF OF STAFF',
    'CHIEF FINANCIAL OFFICER',
    'CHIEF OPERATING OFFICER',
    '^COMMUNICATIONS$',
    'FOOD AND NUTRITION SVC',
    'INNOVATION DEPARTMENT',
    'INSTITUTIONAL ADVANCEMT',
    'LEGAL ADVISOR',
    'PROFESSIONAL DEVELOPMNT',
    'RESEARCH ASSESS AND EVAL',
    'STRATEGY DEPARTMENT'
]

bps_str = '|'.join(bps_phrases)
bps_mask = earnings_df['Department'].str.contains(bps_str, regex=True)

earnings_df.loc[bps_mask,'Program'] = earnings_df.loc[bps_mask,'Department']
earnings_df.loc[bps_mask,'Department'] = 'BOSTON PUBLIC SCHOOLS'

In [17]:
show_unique('Department')

array(['ADMINISTRATION AND FINANCE', 'AGE STRONG',
       'ALIGHIERI MONTESSORI SCHOOL', 'ARTS AND CULTURAL DEVELOPMENT',
       'ASD GRAPHIC ARTS', 'ASD HUMAN RESOURCES',
       'ASD INTERGVERNMTL RELATIONS', 'ASD OFFICE OF BUDGET MANGMNT',
       'ASD OFFICE OF LABOR RELATION', 'ASD PURCHASING DIVISION',
       'ASSESSING DEPARTMENT', 'AUDITING DEPARTMENT',
       'BOSTON CITY COUNCIL', 'BOSTON CNTR - YOUTH AND FAMILIES',
       'BOSTON CNTR-YOUTH AND FAMILIES', 'BOSTON COLLABORATIVE HIGH SCH',
       'BOSTON FIRE DEPARTMENT', 'BOSTON POLICE DEPARTMENT',
       'BOSTON PUBLIC LIBRARY', 'BOSTON PUBLIC SCHOOLS',
       'BOSTON RETIREMENT SYSTEM', 'CEMETERY DIVISION', 'CITY CLERK',
       'COMM FOR PERSONS WITH DISABIL', 'CONSUMER AFFAIRS AND LICENSING',
       'DEPT OF VOTER MOBILIZATION', 'DIPLOMAS PLUS',
       'DND NEIGHBORHOOD DEVELOPMENT', 'DPT OF INNOVATION AND TECHNOLOGY',
       'DUDLEY ST NEIGHBORHOOD SCHOOL', 'ELDERLY COMMISSION',
       'ELECTION DIVISION', 'EMERGENCY MANAGE

In [18]:
#purpose: replace incorrect department names with correct ones when applicable

dept_transforms = {
    '^OFFICE OF FINANCE AND BUDGET$':'ADMINISTRATION AND FINANCE', #*OFFICE OF FINANCE AND BUDGET: 2014 - 2015
    '^OFFICE OF ADMIN AND FINANCE$':'ADMINISTRATION AND FINANCE', 
    '^DND NEIGHBORHOOD DEVELOPMENT$':'NEIGHBORHOOD DEVELOPMENT',
    '^ARTS AND CULTURAL DEVELOPMENT$':'OFFICE OF ARTS AND CULTURE',
    '^ASD GRAPHIC ARTS$':'OFFICE OF ARTS AND CULTURE',
    '^ASD HUMAN RESOURCES$':'HUMAN RESOURCES',
    '^ASD INTERGVERNMTL RELATIONS$':'INTERGOVERNMENTAL RELATIONS',
    '^ASD OFFICE OF BUDGET MANGMNT$':'BUDGET MANAGEMENT',
    '^ASD OFFICE OF LABOR RELATION$':'OFFICE OF LABOR RELATIONS',
    '^ASD PURCHASING DIVISION$':'PROCUREMENT', #*AKA PURCHASING
    '^BOSTON CITY COUNCIL$':'CITY COUNCIL',
    '^BOSTON CNTR \- YOUTH AND FAMILIES$':'BOSTON CENTER FOR YOUTH AND FAMILIES',
    '^BOSTON CNTR\-YOUTH AND FAMILIES$':'BOSTON CENTER FOR YOUTH AND FAMILIES',
    '^BOSTON FIRE DEPARTMENT$':'FIRE DEPARTMENT',
    '^BOSTON POLICE DEPARTMENT$':'POLICE DEPARTMENT',
    '^BOSTON RETIREMENT SYSTEM$':'RETIREMENT DEPARTMENT',
    '^STATE BOSTON RETIREMENT SYST$':'RETIREMENT DEPARTMENT',
    '^COMM FOR PERSONS WITH DISABIL$':'COMMISSION FOR PERSONS WITH DISABILITIES',
    '^DEPT OF VOTER MOBILIZATION$':'ELECTION DEPARTMENT', #*DEPARTMENT OF VOTER MOBILIZATION: <2013
    '^DPT OF INNOVATION AND TECHNOLOGY$':'DEPARTMENT OF INNOVATION AND TECHNOLOGY',
    '^ELDERLY COMMISSION$':'AGE STRONG', #*ELDERLY COMMISSION <2018
    '^ELECTION DIVISION$':'ELECTION DEPARTMENT',
    '^IMMIGRANT ADVANCEMENT$':'OFFICE FOR IMMIGRANT ADVANCEMENT',
    '^INSPECTIONAL SERVICES DEPT$':'INSPECTIONAL SERVICES DEPARTMENT',
    '^OFC CHF PUBLIC WORKS TRANSPORT$':'OFFICE OF STREETS', #*OFFICE OF CHIEF OF PUBLIC WORKS AND TRANSPORT <2013
    '^OFC OF STRTS TRNSP AND SANI$':'OFFICE OF STREETS', #*OFFICE OF STREETS, TRANSPORTATION, AND SANITATION 2014 - 2015
    '^OFFICE OF CIVIL RIGHTS$':'FAIR HOUSING AND EQUITY', #*OFFICE OF CIVIL RIGHTS <2013
    '^OFFICE OF NEW BOSTONIANS$':'OFFICE FOR IMMIGRANT ADVANCEMENT', #*OFFICE OF NEW BOSTONIANS <2017
    '^PARKS DEPARTMENT$':'PARKS AND RECREATION DEPARTMENT',
    '^PROPERTY MANAGEMENT$':'PROPERTY MANAGEMENT DEPARTMENT',
    '^YOUTH FUND$':'YOUTH ENGAGEMENT AND EMPLOYMENT', #*YOUTH FUND <2013
    "^WOMEN'S COMMISSION$":"WOMEN'S ADVANCEMENT", #*WOMEN'S COMMISSION <2013
    '^OPAT$': 'OFFICE OF POLICE ACCOUNTABILITY AND TRANSPARENCY',
    'SERVS$': 'SERVICES'
}

earnings_df['Department'].replace(dept_transforms, regex=True, inplace=True)

In [19]:
show_unique('Department')

array(['ADMINISTRATION AND FINANCE', 'AGE STRONG',
       'ALIGHIERI MONTESSORI SCHOOL', 'ASSESSING DEPARTMENT',
       'AUDITING DEPARTMENT', 'BOSTON CENTER FOR YOUTH AND FAMILIES',
       'BOSTON COLLABORATIVE HIGH SCH', 'BOSTON PUBLIC LIBRARY',
       'BOSTON PUBLIC SCHOOLS', 'BUDGET MANAGEMENT', 'CEMETERY DIVISION',
       'CITY CLERK', 'CITY COUNCIL',
       'COMMISSION FOR PERSONS WITH DISABILITIES',
       'CONSUMER AFFAIRS AND LICENSING',
       'DEPARTMENT OF INNOVATION AND TECHNOLOGY', 'DIPLOMAS PLUS',
       'DUDLEY ST NEIGHBORHOOD SCHOOL', 'ELECTION DEPARTMENT',
       'EMERGENCY MANAGEMENT', 'ENVIRONMENT DEPARTMENT',
       'FAIR HOUSING AND EQUITY', 'FENWAY HIGH SCHOOL',
       'FINANCE COMMISSION', 'FIRE DEPARTMENT', 'GREATER EGLESTON HIGH',
       'HARBOR HIGH', 'HBI RETIREES ET AL', 'HEALTH AND HUMAN SERVICES',
       'HUMAN RESOURCES', 'HUMAN RIGHTS COMMISSION',
       'INSPECTIONAL SERVICES DEPARTMENT', 'INTERGOVERNMENTAL RELATIONS',
       'LAW DEPARTMENT', 'LICENSI

In [20]:
#purpose: some depts should actually be programs, this function can be used to set departments to programs

def replace_dept(original_dept, new_dept, new_program):
    mask = earnings_df['Department'].str.contains(original_dept, regex=True)
    earnings_df.loc[mask,'Program'] = new_program
    earnings_df.loc[mask,'Department'] = new_dept

In [21]:
#use the above function to fix some of the Department data
replace_dept('CEMETERY DIVISION', 'PARKS AND RECREATION DEPARTMENT', 'CEMETERY')
replace_dept('LICENSING BOARD', 'CONSUMER AFFAIRS AND LICENSING', 'LICESNING BOARD')
replace_dept("MAYOR\'S OFFICE\-PUBLIC INFO", "MAYOR'S OFFICE", "MAYOR'S COMMUNICATIONS")
replace_dept('OFC BOSTON RESIDENTS JOB POL', 'OFFICE OF ECONOMIC DEVELOPMENT', 'BOSTON RESIDENTS JOB POLICY OFFICE')
replace_dept('SMALL AND LOCAL BUSINESS', 'OFFICE OF ECONOMIC DEVELOPMENT', 'SMALL AND LOCAL BUSINESS')
replace_dept('TRAFFIC DIVISION', 'TRANSPORTATION DEPARTMENT', 'TRAFFIC DIVISION')
replace_dept('TRANSPORTATION-PARKING CLERK', 'TRANSPORTATION DEPARTMENT', 'PARKING CLERK')
replace_dept('TREASURY-COLLECTING DIVISION', 'TREASURY DEPARTMENT', 'TREASURY DIVISION')
replace_dept('TREASURY-TREASURY DIVISION', 'TREASURY DEPARTMENT', 'COLLECTING DIVISION')
replace_dept('WORKERS COMPENSATION SERVICE', 'HUMAN RESOURCES', 'WORKERS COMP')
replace_dept('HBI RETIREES ET AL', 'HUMAN RESOURCES', 'HEALTH BENEFITS AND INSURANCE')

In [22]:
show_unique('Department')

array(['ADMINISTRATION AND FINANCE', 'AGE STRONG',
       'ALIGHIERI MONTESSORI SCHOOL', 'ASSESSING DEPARTMENT',
       'AUDITING DEPARTMENT', 'BOSTON CENTER FOR YOUTH AND FAMILIES',
       'BOSTON COLLABORATIVE HIGH SCH', 'BOSTON PUBLIC LIBRARY',
       'BOSTON PUBLIC SCHOOLS', 'BUDGET MANAGEMENT', 'CITY CLERK',
       'CITY COUNCIL', 'COMMISSION FOR PERSONS WITH DISABILITIES',
       'CONSUMER AFFAIRS AND LICENSING',
       'DEPARTMENT OF INNOVATION AND TECHNOLOGY', 'DIPLOMAS PLUS',
       'DUDLEY ST NEIGHBORHOOD SCHOOL', 'ELECTION DEPARTMENT',
       'EMERGENCY MANAGEMENT', 'ENVIRONMENT DEPARTMENT',
       'FAIR HOUSING AND EQUITY', 'FENWAY HIGH SCHOOL',
       'FINANCE COMMISSION', 'FIRE DEPARTMENT', 'GREATER EGLESTON HIGH',
       'HARBOR HIGH', 'HEALTH AND HUMAN SERVICES', 'HUMAN RESOURCES',
       'HUMAN RIGHTS COMMISSION', 'INSPECTIONAL SERVICES DEPARTMENT',
       'INTERGOVERNMENTAL RELATIONS', 'LAW DEPARTMENT', "MAYOR'S OFFICE",
       'NAN', 'NEIGHBORHOOD DEVELOPMENT', 'NEIG

In [23]:
#preliminary cleaning for program column
earnings_df['Program'] = earnings_df['Program'].replace({'^BPS':'', 'BOSTON PUBLIC SCHOOLS':'', '\"':''}, regex=True).str.strip()
earnings_df['Program'] = earnings_df['Program'].replace(np.nan, '')
show_unique('Program')

array(['', 'ACCOUNTABILITY', 'ACHIEVEMENT GAP', 'ADAMS ELEMENTARY',
       'ADULT EDUCATION', 'ADVANCEMENT AND EXT AFFAIRS',
       'ALTERNATIVE EDUCATION', 'ALTERNATIVE EDUCATION HS',
       'ANOTHER COURSE TO COLLEG', 'ASST SUPERINTENDENT-NETWORK A',
       'ASST SUPERINTENDENT-NETWORK B', 'ASST SUPERINTENDENT-NETWORK C',
       'ASST SUPERINTENDENT-NETWORK D', 'ASST SUPERINTENDENT-NETWORK E',
       'ASST SUPERINTENDENT-NETWORK F', 'ASST SUPERINTENDENT-NETWORK G',
       'ATHLETICS', 'BALDWIN ELC', 'BATES ELEMENTARY',
       'BEETHOVEN ELEMENTARY', 'BI-WEEKLY 26 LEAVE',
       'BLACKSTONE ELEMENTARY', 'BOSTON ARTS ACADEMY',
       'BOSTON COMM LEADERSHIP AC', 'BOSTON EVENING ACADEMY',
       'BOSTON INTERNATIONAL HS', 'BOSTON LATIN',
       'BOSTON MIDDLE SCHOOL ACADEMY',
       'BOSTON RESIDENTS JOB POLICY OFFICE', 'BOSTON SCHOOL COMMITTEE',
       'BRADLEY ELEMENTARY', 'BRIGHTON HIGH', 'BTU PILOT', 'BURKE HIGH',
       'BUSINESS SERVICE', 'CAPITAL PLANNING UNIT',
       'CAREER AN

In [24]:
program_transforms = {
    'MGMT':' MANAGEMENT ',
    'EXT AFFAIRS':' EXTERNAL AFFAIRS ',
    'CL10':' ',
    'COM ACD':' COMMUNITY ACADEMY OF ',
    'P A SHAW':' PA SHAW ',
    'NURS':' NURSES ',
    'WEST ROXBURY HIGH':' WEST ROXBURY ACADEMY ', 
    'WREC\:':' ',
    'HPEC\:':' ',
    'MC CORMACK MIDDLE':' MCCORMACK MIDDLE ', 
    'KENNEDY EM':' EDWARD M KENNEDY ',
    'KENNEDY JF':' JOHN F KENNEDY ',
    'KENNEDY PJ':' PATRICK J KENNEDY ',
    'GREENWOOD S':' SARAH GREENWOOD ',
    'MC KINLEY MIDDLE':' MCKINLEY MIDDLE ',
    'HORACE MANN':'HORACE MANN SCHOOL',
    'WITHTHROP':' WINTHROP ',
    'SOUTH BOSTON HS - EXCEL':' EXCEL HIGH SCHOOL ',
    'MPHCOMMERCE':' MADISON PARK HIGH SCHOOL - COMMERCE ',
    'MPHCRAFTS':' MADISON PARK HIGH SCHOOL - CRAFTS ',
    'MPHHEALTH':' MADISON PARK HIGH SCHOOL - HEALTH ',
    'MPHFRESHMAN':' MADISON PARK HIGH SCHOOL - FRESHMAN ',
    'FACILITY MANAGEMENT AND A AND R':' FACILITIES MANAGEMENT ',
    'FACILITY MANAGEMENT':' FACILITIES MANAGEMENT ',
    'FACILITITES MANAGEMENT':' FACILITIES MANAGEMENT ',
    'GREENWOOD E':' ELIHU GREENWOOD ',
    'ALTERNATIVE EDUCATION HIGH SCHOOL':' ALTERNATIVE EDUCATION ',
    'COLLEG':' COLLEGE ',
    'SCH':' SCHOOL ',  
    'INTERVTN CT':' INTERVENTION CENTER ',
    'COUSELING':' COUNSELING ',
    'SERVICE':' SERVICES ',
    'SER':' SERVICES ',
    'HI':' HIGH ',
    'ED':' EDUCATION ',
    'HS':' HIGH SCHOOL ',
    'ACAD':' ACADEMY ',
    'SVC':' SERVICES ',
    'ASST':' ASSISTANT ',
    'COMM':' COMMUNITY ',
    'AC':' ACADEMY ',
    'ST':' STREET ',
    'HEARING AND APPEAL AND ATTENDANC':' HEARINGS AND APPEALS ',
    'KITCHEN FOOD':' KITCHEN AND FOOD ',
    'LEARN':' LEARNING ',
    'FAM AND STUDENT ENGAGEMT':' FAMILY AND STUDENT ENGAGEMENT ',
    'ADVANCEMT':' ADVANCEMENT ',
    'ACCOUNTABILITY':' TURNAROUND AND TRANSFORMATION ',
    'RESEARCH ASSESS AND EVAL':' DATA AND ACCOUNTABILITY ',
    'TRANFORMATION':' TRANSFORMATION ',
    'ELEMENTARY$':' ELEMENTARY SCHOOL',
    'MIDDLE$':' MIDDLE SCHOOL',
    'HIGH$':' HIGH SCHOOL',
    'HIGH SCH$': ' HIGH SCHOOL',
    'PILOT$':' PILOT SCHOOL',
    '\-NETWORK':' - NETWORK',
    'INFO AND INSTR TECHNOLOGY': 'INFORMATION AND INSTRUCTIONAL TECHNOLOGY'
}

for key in program_transforms:
    new_key = '(^|[^a-zA-Z0-9])'+key+'([[^a-zA-Z0-9]]|$)'
    earnings_df['Program'] = earnings_df['Program'].replace(new_key, program_transforms[key], regex=True).str.strip()
    

c:\users\cpsch\gitrepos\bostonsalaries\venv\lib\site-packages\pandas\core\dtypes\inference.py:213: FutureWarning: Possible nested set at position 22
  re.compile(obj)
c:\users\cpsch\gitrepos\bostonsalaries\venv\lib\site-packages\pandas\core\dtypes\inference.py:213: FutureWarning: Possible nested set at position 29
  re.compile(obj)
c:\users\cpsch\gitrepos\bostonsalaries\venv\lib\site-packages\pandas\core\dtypes\inference.py:213: FutureWarning: Possible nested set at position 25
  re.compile(obj)
c:\users\cpsch\gitrepos\bostonsalaries\venv\lib\site-packages\pandas\core\dtypes\inference.py:213: FutureWarning: Possible nested set at position 26
  re.compile(obj)
c:\users\cpsch\gitrepos\bostonsalaries\venv\lib\site-packages\pandas\core\dtypes\inference.py:213: FutureWarning: Possible nested set at position 35
  re.compile(obj)
c:\users\cpsch\gitrepos\bostonsalaries\venv\lib\site-packages\pandas\core\dtypes\inference.py:213: FutureWarning: Possible nested set at position 24
  re.compile(obj

In [25]:
show_unique('Program')

array(['', 'ACHIEVEMENT GAP', 'ADAMS ELEMENTARY SCHOOL',
       'ADULT EDUCATION', 'ADVANCEMENT AND EXTERNAL AFFAIRS',
       'ALTERNATIVE EDUCATION', 'ALTERNATIVE EDUCATION HIGH SCHOOL',
       'ANOTHER COURSE TO COLLEGE', 'ASST SUPERINTENDENT-NETWORK A',
       'ASST SUPERINTENDENT-NETWORK B', 'ASST SUPERINTENDENT-NETWORK C',
       'ASST SUPERINTENDENT-NETWORK D', 'ASST SUPERINTENDENT-NETWORK E',
       'ASST SUPERINTENDENT-NETWORK F', 'ASST SUPERINTENDENT-NETWORK G',
       'ATHLETICS', 'BALDWIN ELC', 'BATES ELEMENTARY SCHOOL',
       'BEETHOVEN ELEMENTARY SCHOOL', 'BI-WEEKLY 26 LEAVE',
       'BLACKSTONE ELEMENTARY SCHOOL', 'BOSTON ARTS ACADEMY',
       'BOSTON COMM LEADERSHIP ACADEMY', 'BOSTON EVENING ACADEMY',
       'BOSTON INTERNATIONAL HIGH SCHOOL', 'BOSTON LATIN',
       'BOSTON MIDDLE SCHOOL ACADEMY',
       'BOSTON RESIDENTS JOB POLICY OFFICE', 'BOSTON SCHOOL COMMITTEE',
       'BRADLEY ELEMENTARY SCHOOL', 'BRIGHTON HIGH SCHOOL',
       'BTU PILOT SCHOOL', 'BURKE HIGH SCHO

In [26]:
earnings_df.sample(100)

,Department,Title,Regular,Retro,Other,Overtime,Injury,Total,Postal,Year,Last,First,Program
19705,TRANSPORTATION DEPARTMENT,PARKING METER SUPERVISOR,47895.80,0.00,0.00,321.62,0.00,48217.42,UNKNOWN,2013,STEVENS,KIMBERLY A,
9495,BOSTON PUBLIC SCHOOLS,SENIOR CUSTODIAN C,50694.20,0.00,0.00,4468.73,0.00,55162.93,UNKNOWN,2012,MCGLOTHIN,HENRY J,
19366,PARKS AND RECREATION DEPARTMENT,GOLF COURSE OPERATIONS MANAGER,47956.08,532.53,849.42,0.00,0.00,49338.03,02124,2018,BARTON,WILLIAM,
15171,BOSTON PUBLIC SCHOOLS,TEACHER,96104.43,0.00,4500.00,0.00,0.00,100604.43,02090,2015,O'CONNOR,CHERYL A,RUSSELL ELEMENTARY SCHOOL
12213,BOSTON PUBLIC SCHOOLS,ANALYST (C),110037.98,0.00,1500.00,0.00,0.00,111537.98,02124,2015,MACRAE,YVONNE,OFFICE OF BUDGET MANAGEMENT
8864,BOSTON PUBLIC SCHOOLS,SUBSTITUTE CAFETERIA ATTENDANT,4317.75,0.00,0.00,0.00,0.00,4317.75,UNKNOWN,2011,BAKER,ALEXANDRIA SB,
18944,POLICE DEPARTMENT,POLICE DISPATCHER,41324.90,0.00,5798.00,0.00,0.00,47122.90,UNKNOWN,2014,LI,WEI,
7978,BOSTON PUBLIC SCHOOLS,TEACHER,105966.48,0.00,0.00,0.00,0.00,105966.48,02351,2021,MCCARTHY,DAVID F,TECH BOSTON ACADEMY
8483,BOSTON PUBLIC SCHOOLS,ANALYST (C),102343.80,0.00,0.00,0.00,0.00,102343.80,02125,2021,BURNETT,TASHEENA MARIE,DATA AND ACCOUNTABILITY
13126,BOSTON PUBLIC SCHOOLS,SURROUND CARE PARAPROFESSIONAL,30463.39,0.00,1600.00,0.00,0.00,32063.39,UNKNOWN,2012,KING,JIMMY TYRONE,


In [27]:
earnings_df.size

3170297

In [28]:
#purpose: verify/retrieve cabinet and department names using operating budget data
#TODO: Replace with 2021 budget CSV?
    
budget_df = pd.read_csv('./data/operating_budget.csv', encoding='latin1')
budget_df

,Cabinet,Dept,Program,Expense Category,FY18 Actual,FY19 Actual,20 Budget,21 Budget
0,Mayors Cabinet,Mayor's Office,Mayor's Administration,Personnel Services,"1,185,285","1,400,773","1,529,111","1,845,647"
1,Mayors Cabinet,Mayor's Office,Mayor's Administration,Contractual Services,"179,041","85,769","83,334","74,334"
2,Mayors Cabinet,Mayor's Office,Mayor's Administration,Supplies & Materials,"53,822","37,539","37,183","37,183"
3,Mayors Cabinet,Mayor's Office,Mayor's Administration,Current Charges & Obligations,"7,543","8,801","7,213","7,213"
4,Mayors Cabinet,Mayor's Office,Mayor's Administration,Equipment,"17,709","17,273","25,000","25,000"
...,...,...,...,...,...,...,...,...
1321,Other,Risk Retention Reserve,Risk Retention Reserve,Other Expenses,"3,000,000","3,000,000","3,000,000","3,000,000"
1322,Other,Tax Title,Tax Title,Other Expenses,-,-,"600,000","600,000"
1323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
transforms = {
    'LIBRARY DEPARTMENT':'BOSTON PUBLIC LIBRARY', 
    'COMMISSION FOR PERSONS W/DISABILITIES':'COMMISSION FOR PERSONS WITH DISABILITIES', 
    'LICENSING_BOARD':'LICENSING BOARD',
    'BOSTON VETS':"VETERANS' SERVICES"
}
budget_df = budget_df[['Cabinet','Dept']].drop_duplicates().apply(lambda x: x.str.upper())
budget_df = budget_df.replace({'\.':' ', '\&':' AND ', '\,':' ', '\s+':' '}, regex=True).apply(lambda x: x.str.strip())
budget_df.replace(transforms, inplace=True)

In [30]:
earnings_df = earnings_df.join(budget_df.set_index('Dept'), how='left', on='Department')
earnings_df = earnings_df[['First','Last','Year','Cabinet','Department','Program','Title','Regular','Retro','Overtime','Injury','Other','Total','Postal']]

In [31]:
earnings_df.replace('', np.nan, inplace=True)

In [32]:
earnings_df.sort_values(['Last', 'First', 'Year']).head(500)

,First,Last,Year,Cabinet,Department,Program,Title,Regular,Retro,Overtime,Injury,Other,Total,Postal
19168,BRIAN,A'HEARN,2018,HEALTH AND HUMAN SERVICES,BOSTON CENTER FOR YOUTH AND FAMILIES,NaN,BUILDING ASSISTANT,24326.46,0.00,1230.40,0.00,0.00,25556.86,02132
16596,BRIAN,A'HEARN,2019,HEALTH AND HUMAN SERVICES,BOSTON CENTER FOR YOUTH AND FAMILIES,NaN,BUILDING ASSISTANT,37034.48,0.00,0.00,0.00,0.00,37034.48,02132
16318,BRIAN,A'HEARN,2020,HEALTH AND HUMAN SERVICES,BOSTON CENTER FOR YOUTH AND FAMILIES,NaN,BUILDING ASSISTANT,40042.17,0.00,562.96,0.00,0.00,40605.13,02132
16367,BRIAN,A'HEARN,2021,HEALTH AND HUMAN SERVICES,BOSTON CENTER FOR YOUTH AND FAMILIES,NaN,BUILDING ASSISTANT,41188.54,0.00,738.00,0.00,0.00,41926.54,02132
22053,VANESSA JENNIFER,AARONS,2021,HEALTH AND HUMAN SERVICES,BOSTON CENTER FOR YOUTH AND FAMILIES,NaN,COUNSELOR (CC),2033.54,0.00,0.00,0.00,0.00,2033.54,02115
8362,RUBY A,ABABIO-FERNANDEZ,2011,EDUCATION,BOSTON PUBLIC SCHOOLS,NaN,PRINCIPAL ELEMENTARY,119578.40,0.00,0.00,0.00,0.00,119578.40,UNKNOWN
14675,RUBY A,ABABIO-FERNANDEZ,2012,EDUCATION,BOSTON PUBLIC SCHOOLS,NaN,PRINCIPAL ELEMENTARY,87161.71,0.00,0.00,0.00,12122.88,99284.59,UNKNOWN
0,RUBY A,ABABIO-FERNANDEZ,2013,EDUCATION,BOSTON PUBLIC SCHOOLS,NaN,PRINCIPAL ELEMENTARY,0.00,0.00,0.00,0.00,12122.88,12122.88,UNKNOWN
0,KIDANI A,ABADI,2011,ADMINISTRATION AND FINANCE,ASSESSING DEPARTMENT,NaN,PROPERTY OFFICER (ASN),33065.38,0.00,379.49,0.00,0.00,33444.87,UNKNOWN
240,KIDANI A,ABADI,2012,ADMINISTRATION AND FINANCE,ASSESSING DEPARTMENT,NaN,PROPERTY OFFICER (ASN),34554.41,157.84,0.00,0.00,918.06,35630.31,UNKNOWN


In [33]:
def remove_nulls(d):
    return {k: v for k, v in d.items() if v is not None}

#convert the dataframe into JSON, with one document for each employee and year
j = json.loads(earnings_df.sort_values(['Last','First','Year']).to_json(orient='records'), object_hook=remove_nulls)

In [34]:
client = MongoClient()
db = client['boston_salaries']
collection = db['salaries']

collection.delete_many({})

collection.insert_many(j)

In [35]:
for val in collection.find({'$and':[{'First':'RICHARD','Last':'BECKERS'}]}):
    print(val)

{'_id': ObjectId('6362c3ab8b3fccced1e8c021'), 'First': 'RICHARD', 'Last': 'BECKERS', 'Year': 2021, 'Cabinet': 'PUBLIC SAFETY', 'Department': 'POLICE DEPARTMENT', 'Title': 'POLICE OFFICER', 'Regular': 0.0, 'Retro': 0.0, 'Overtime': 0.0, 'Injury': 0.0, 'Other': 1264843.63, 'Total': 1264843.63, 'Postal': '02119'}
